# LLM from Scratch
Following the below yt playlist: https://youtube.com/playlist?list=PLPTV0NXA_ZSgsLAr8YCgCwhPIJNNtexWu&si=ZfM1-2Pnye5eLXas

Reading in a short story as text sample into Python

## Step 1: Creating Tokens

In [3]:
with open("data files/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of char:",len(raw_text))
print(raw_text[:90])

Total number of char: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so i


In [4]:
import re
text = "hello, world. This, is for testing purposes"
result = re.split(r'([,.]|\s)', text)
print(result)

result = [item for item in result if item.strip()]
print(result, end="\n\n")

text = "hello, world. This!, is for testing --purposes?"
result = re.split(r'([,.-?!:;_"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'for', ' ', 'testing', ' ', 'purposes']
['hello', ',', 'world', '.', 'This', ',', 'is', 'for', 'testing', 'purposes']

['hello', ',', 'world', '.', 'This', '!', ',', 'is', 'for', 'testing', '--', 'purposes', '?']


In [5]:
# preprocessed = re.split(r'([,.?;:\'"()!_-]|--|\s)', raw_text)
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:15])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow']


In [6]:
len(preprocessed)

4690

## Step 2: Creating Token ID

In [7]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [8]:
vocab = {
    token:integer for integer, token in enumerate(all_words)
}

for i, item in enumerate(vocab.items()):
    print(item)
    if i>10: break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)


In [9]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [12]:
tokenizer = SimpleTokenizerV1(vocab)
text = '''"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride.'''
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [13]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [14]:
text = "Hello, do you like tea?"
tokenizer.encode(text)

KeyError: 'Hello'

## Adding special context tokens
- adding <|unk|> and <|endoftext|> tokens 

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
len(vocab.items())

1132

In [21]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [23]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item  in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [24]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea ?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

ids = tokenizer.encode(text)
print(ids)

print(tokenizer.decode(ids))

Hello, do you like tea ? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
